In [1]:
import pandas as pd
import re
from meteostat import Hourly, Point
from datetime import datetime

In [12]:
# Load the CSV file with province centroids
file_path = "belgium_provinces_centroids.csv"  # Change this if needed
locations_df = pd.read_csv(file_path)

# Function to extract latitude and longitude from 'POINT (lon lat)'
def extract_coordinates(point_str):
    match = re.match(r"POINT \(([-\d.]+) ([-\d.]+)\)", point_str)
    if match:
        lon, lat = float(match.group(1)), float(match.group(2))
        return lat, lon
    return None, None  # Return None if parsing fails

# Apply function to extract coordinates
locations_df[['Latitude', 'Longitude']] = locations_df['centroid'].apply(lambda x: pd.Series(extract_coordinates(x)))

# Define start and end dates for weather data
start_date = datetime(2014, 1, 1)
end_date = datetime(2025, 1, 1)

# Create an empty DataFrame for all weather data
all_data = pd.DataFrame()

# Loop through each province and fetch hourly weather data
for _, row in locations_df.iterrows():
    lat, lon = row["Latitude"], row["Longitude"]
    province = row["NAME_2"]  # Province name
    
    if pd.notna(lat) and pd.notna(lon):  # Ensure valid coordinates
        print(f"Fetching data for {province} ({lat}, {lon})...")
        location = Point(lat, lon)
        data = Hourly(location, start=start_date, end=end_date).fetch()
        
        # Add province and coordinate info
        data["Province"] = province
        
        # Append to main DataFrame
        all_data = pd.concat([all_data, data])

# Save final dataset to CSV
output_file = "hourly_weather_data.csv"
all_data.to_csv(output_file, index=True)

print(f"✅ Hourly weather data successfully saved to {output_file}!")

Fetching data for Bruxelles (50.83642580516115, 4.367414882050033)...
Fetching data for Antwerpen (51.231900712244745, 4.721043779498864)...
Fetching data for Limburg (50.98831159244354, 5.4357209224443475)...
Fetching data for Oost-Vlaanderen (51.03629347952689, 3.8188605810442007)...
Fetching data for Vlaams Brabant (50.87303943005528, 4.59072072950776)...
Fetching data for West-Vlaanderen (51.01019740435232, 3.0620446668608117)...
Fetching data for Brabant Wallon (50.66605965845173, 4.589736973914637)...
Fetching data for Hainaut (50.46387637972707, 3.9681364929631004)...
Fetching data for Liège (50.518876085615986, 5.7372263688290746)...
Fetching data for Luxembourg (49.95849015160492, 5.516735682549194)...
Fetching data for Namur (50.252686235186815, 4.862346452029854)...
✅ Hourly weather data successfully saved to hourly_weather_data.csv!
